# Kerasを用いた画像分類入門（３）

##  過学習の抑制

## Google Colab用の設定

Google Colaboratory で実行する場合には下記のセルを実行してください。

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

In [0]:
!unzip /content/gdrive/My\ Drive/GD_share01/noodles.zip -d /content

## ライブラリのインポート

In [0]:
import tensorflow.keras as keras

## 画像データの準備

 解凍した教師画像からデータを読み込むImageDataGeneratorを作成します。  
 下記の訓練用のImageDataGeneratorの初期化パラメータを設定してください。

In [0]:
train_dir = './noodles/train/'
validation_dir = './noodles/validaiton/'
test_dir = './noodles/test/'
image_size = (128, 128)
batch_size = 32
steps_per_epoch = 300 // batch_size
validation_steps = 100 // batch_size
test_steps = 100 // batch_size

augmented_data_generator = keras.preprocessing.image.ImageDataGenerator(rescale=1/255.0,
                                                                       rotation_range=,
                                                                       width_shift_range=,
                                                                       height_shift_range=,
                                                                       brightness_range=,
                                                                       shear_range=,
                                                                       zoom_range=,
                                                                       horizontal_flip=,
                                                                       vertical_flip=)

rescaled_data_generator = keras.preprocessing.image.ImageDataGenerator(rescale=1/255.0)

train_generator = augmented_data_generator.flow_from_directory(train_dir,
                                                           target_size=image_size, 
                                                           batch_size=batch_size)
validation_generator = rescaled_data_generator.flow_from_directory(validation_dir,
                                                                target_size=image_size, 
                                                                batch_size=batch_size)
test_generator = rescaled_data_generator.flow_from_directory(test_dir,
                                                                target_size=image_size, 
                                                                batch_size=batch_size)

## ニューラルネットワークの過学習とは？

###  モデルの構築

Conv2DとMaxPooling2Dを用いた画像の２値分類のための畳み込みニューラルネットワークを構成します。

In [0]:
model = keras.models.Sequential()

model.add(keras.layers.Conv2D(32, (3, 3), strides=(1, 1), activation='relu', input_shape=(128, 128, 3)))
model.add(keras.layers.MaxPooling2D((2, 2)))
model.add(keras.layers.Conv2D(64, (3, 3), strides=(1, 1), activation='relu'))
model.add(keras.layers.MaxPooling2D((2, 2)))
model.add(keras.layers.Conv2D(128, (3, 3), strides=(1, 1), activation='relu'))
model.add(keras.layers.MaxPooling2D((2, 2)))
model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(128, activation='relu'))
model.add(keras.layers.Dense(2, activation='softmax'))


### モデル構造の表示

In [0]:
model.summary()

### モデルのコンパイル

モデルをコンパイルします。

In [0]:
model.compile(loss='categorical_crossentropy', 
              optimizer='adam',
              metrics=['acc'])

### モデルの訓練

ここでは30エポック、モデルの訓練を行います。

In [0]:
history = model.fit_generator(train_generator,
                             epochs=50,
                             steps_per_epoch=steps_per_epoch,
                             validation_data=validation_generator,
                             validation_steps=validation_steps)

### 学習経過のグラフ化

下記のプログラムで、エポックごとの損失と正解率をグラフ化できます。  
グラフから何が読み取れるでしょうか？

In [0]:
import matplotlib.pyplot as plt
%matplotlib inline

acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(acc))

plt.plot(epochs, acc, label='Train Accuracy')
plt.plot(epochs, val_acc, label='Validation Accuracy')
plt.legend()
plt.show()

plt.plot(epochs, loss, label='Train Loss')
plt.plot(epochs, val_loss, label='Validation Loss')
plt.legend()
plt.show()


### モデルの評価

In [0]:
evaluation = model.evaluate_generator(test_generator, steps=test_steps)
print(evaluation)

最も性能の良かったモデルを読み込みます。

## モデルによる予測

下記のプログラムで、学習したモデルを使ってテスト用データの分類（予測）を行うことができます。  
予測結果がどのような形式なのかを確認してください。

In [0]:
predictions = model.predict_generator(test_generator, steps=test_steps)

print(predictions)

## モデルの予測結果を見てみる

次のプログラムを実行すると、テスト用データの１バッチ分についてモデルの予測値と正解を画像つきで確認できます。  
どのような結果になったかを確認してください。

In [0]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

code_to_class = {0: 'ラーメン', 1: 'うどん'}

x, y = next(test_generator)
y_true = np.argmax(y, axis=1)
y_prob = model.predict_on_batch(x)
y_pred = np.argmax(y_prob, axis=1)

for i in range(len(x)):
    plt.imshow(x[i])
    plt.show()
    print('推定：', code_to_class[y_pred[i]])
    print('正解：', code_to_class[y_true[i]])